In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers, Sequential, callbacks, models
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import os
import warnings
from sklearn.exceptions import FitFailedWarning
from keras.wrappers.scikit_learn import KerasClassifier

warnings.simplefilter('always', FitFailedWarning)

seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


def train_test_split(data, train_split=.7, val_split=.26, test_split=.06, shuffle=True, shuffle_size=10000):
    data_size = len(data)
    if shuffle:
        data = data.shuffle(shuffle_size, seed=17)

    train_size = int(train_split * data_size)
    val_size = int(val_split * data_size)

    train_data = data.take(train_size)
    val_data = data.skip(train_size).take(val_size)
    test_data = data.skip(train_size).skip(val_size)

    return train_data, val_data, test_data


def predicts(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    predicted_class = classes[np.argmax(predictions[0])]
    confidence = round(100 * np.max(predictions[0]), 2)
    return predicted_class, confidence


def create_model(optimizer='adam', kernel_size=(3, 3), pool_size=(2, 2), activation='relu'):
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=kernel_size, activation=activation, input_shape=input_shape),
        layers.MaxPooling2D(pool_size),
        layers.Conv2D(64, kernel_size=kernel_size, activation=activation),
        layers.MaxPooling2D(pool_size),
        layers.Conv2D(64, kernel_size=kernel_size, activation=activation),
        layers.MaxPooling2D(pool_size),
        layers.Conv2D(64, kernel_size=kernel_size, activation=activation),
        layers.MaxPooling2D(pool_size),
        layers.Conv2D(64, kernel_size=kernel_size, activation=activation),
        layers.MaxPooling2D(pool_size),
        layers.Conv2D(64, kernel_size=kernel_size, activation=activation),
        layers.MaxPooling2D(pool_size),
        layers.Flatten(),
        layers.Dense(64, activation=activation),
        layers.Dense(len(classes), activation="softmax")
    ])

    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=["accuracy"])
    return model


def plot_model_accuracy(history_):
    acc = history_.history['accuracy']
    val_acc = history_.history['val_accuracy']
    loss = history_.history['loss']
    val_loss = history_.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, label='Training acc')
    plt.plot(epochs, val_acc, label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()


data_dir = "../input/sport-celebrity-image-classification/Sports-celebrity images"
img_size = 256
batch_size = 32
df = tf.keras.preprocessing.image_dataset_from_directory(directory=data_dir,
                                                         shuffle=True,
                                                         seed=seed_value,
                                                         image_size=(img_size, img_size),
                                                         batch_size=batch_size)
classes = df.class_names

all_files = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        all_files.append(os.path.join(root, file))

class_counts = {}
for file_path in all_files:
    class_name = os.path.basename(os.path.dirname(file_path))
    if class_name not in class_counts:
        class_counts[class_name] = 1
    else:
        class_counts[class_name] += 1

plt.figure(figsize=(12, 6))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))

data_count = len(df)
print(f"Numbers for data: {data_count}")

for image_batch, label_batch in df.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

plt.figure(figsize=(8, 8))
for image_batch, label_batch in df.take(1):
    for i in range(12):
        ax = plt.subplot(4, 3, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(classes[label_batch[i]])
        plt.axis("off")
plt.show()

train_data, val_data, test_data = train_test_split(df)

print("Len of train:{}".format(len(train_data)))
print("Len of val:{}".format(len(val_data)))
print("Len of test:{}".format(len(test_data)))

datagen = ImageDataGenerator(rotation_range=50,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             brightness_range=[.2, .5])

for X_batch, y_batch in train_data:
    for i in range(3):
        augmented_images, augmented_labels = next(datagen.flow(X_batch, y_batch, batch_size=3))
        X_batch = np.concatenate((X_batch, augmented_images), axis=0)
        y_batch = np.concatenate((y_batch, augmented_labels), axis=0)

train_data = train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_data = val_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

channels = 3
input_shape = (img_size, img_size, channels)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(.5),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(.5),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(len(classes), activation="softmax")
])

model.build(input_shape=input_shape)

model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=["accuracy"])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(train_data,
                    epochs=200,
                    verbose=1,
                    batch_size=batch_size,
                    validation_data=val_data,
                    callbacks=[early_stopping])

scores = model.evaluate(test_data)

plot_model_accuracy(history)

plt.figure(figsize=(15, 15))
for images, labels in test_data.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predicts(model, images[i].numpy())
        actual_class = classes[labels[i]]
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class},\n Confidence: {confidence}%")
        plt.axis("off")

actual_list = []
pred_list = []
for X_batch, y_batch in test_data:
    y_batch = y_batch.numpy().tolist()
    y_pred = model.predict(X_batch)
    for i in y_batch:
        actual_list.append(i)
    for j in y_pred:
        pred_list.append(np.argmax(j, axis=-1).tolist())

cm = confusion_matrix(actual_list, pred_list)
sns.heatmap(cm, annot=True, cmap='seismic', fmt='g')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.show()

X_train = []
y_train = []
for X_batch, y_batch in train_data:
    X_train.append(X_batch.numpy())
    y_train.append(y_batch.numpy())
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

warnings.filterwarnings('ignore')

optimizers = ['rmsprop', 'adam']
kernel_size = [(3, 3), (4, 4)]
pool_size = [(2, 2), (3, 3)]
activation = ['relu', 'sigmoid', 'tanh']
epochs = [60]

param_grid = dict(optimizer=optimizers,
                  kernel_size=kernel_size,
                  pool_size=pool_size,
                  activation=activation,
                  epochs=epochs)

model = KerasClassifier(build_fn=create_model, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1, cv=3).fit(X_train, y_train)

print("The best score: %f using %s" % (grid.best_score_, grid.best_params_))

channels = 3
input_shape = (img_size, img_size, channels)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="tanh", input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="tanh"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(.5),
    layers.Conv2D(64, (3, 3), activation="tanh"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="tanh"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(.5),
    layers.Conv2D(64, (3, 3), activation="tanh"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="tanh"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="tanh"),
    layers.Dense(len(classes), activation="softmax")
])

model.build(input_shape=input_shape)

model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=["accuracy"])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=30)

history = model.fit(train_data,
                    epochs=300,
                    verbose=1,
                    batch_size=batch_size,
                    validation_data=val_data,
                    callbacks=[early_stopping])

scores = model.evaluate(test_data)

plot_model_accuracy(history)

plt.figure(figsize=(15, 15))
for images, labels in test_data.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predicts(model, images[i].numpy())
        actual_class = classes[labels[i]]
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class},\n Confidence: {confidence}%")
        plt.axis("off")

actual_list = []
pred_list = []
for X_batch, y_batch in test_data:
    y_batch = y_batch.numpy().tolist()
    y_pred = model.predict(X_batch)
    for i in y_batch:
        actual_list.append(i)
    for j in y_pred:
        pred_list.append(np.argmax(j, axis=-1).tolist())

cm = confusion_matrix(actual_list, pred_list)
sns.heatmap(cm, annot=True, cmap='seismic', fmt='g')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.show()

X_train = []
y_train = []
for X_batch, y_batch in train_data:
    X_train.append(X_batch.numpy())
    y_train.append(y_batch.numpy())
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

warnings.filterwarnings('ignore')

optimizers = ['rmsprop', 'adam']
kernel_size = [(3, 3), (4, 4)]
pool_size = [(2, 2), (3, 3)]
activation = ['relu', 'sigmoid', 'tanh']
epochs = [60]

param_grid = dict(optimizer=optimizers,
                  kernel_size=kernel_size,
                  pool_size=pool_size,
                  activation=activation,
                  epochs=epochs)

model = KerasClassifier(build_fn=create_model, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1, cv=3).fit(X_train, y_train)

print("The best score: %f using %s" % (grid.best_score_, grid.best_params_))

channels = 3
input_shape = (img_size, img_size, channels)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(len(classes), activation="softmax")
])

model.build(input_shape=input_shape)

model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=["accuracy"])

history = model.fit(train_data,
                    epochs=200,
                    verbose=1,
                    batch_size=batch_size,
                    validation_data=val_data)

scores = model.evaluate(test_data)

plot_model_accuracy(history)

plt.figure(figsize=(15, 15))
for images, labels in test_data.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predicts(model, images[i].numpy())
        actual_class = classes[labels[i]]
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class},\n Confidence: {confidence}%")
        plt.axis("off")

actual_list = []
pred_list = []
for X_batch, y_batch in test_data:
    y_batch = y_batch.numpy().tolist()
    y_pred = model.predict(X_batch)
    for i in y_batch:
        actual_list.append(i)
    for j in y_pred:
        pred_list.append(np.argmax(j, axis=-1).tolist())

cm = confusion_matrix(actual_list, pred_list)
sns.heatmap(cm, annot=True, cmap='seismic', fmt='g')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.show()

X_train = []
y_train = []
for X_batch, y_batch in train_data:
    X_train.append(X_batch.numpy())
    y_train.append(y_batch.numpy())
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

warnings.filterwarnings('ignore')

optimizers = ['rmsprop', 'adam']
kernel_size = [(3, 3), (4, 4)]
pool_size = [(2, 2), (3, 3)]
activation = ['relu', 'sigmoid', 'tanh']
epochs = [60]

param_grid = dict(optimizer=optimizers,
                  kernel_size=kernel_size,
                  pool_size=pool_size,
                  activation=activation,
                  epochs=epochs)

model = KerasClassifier(build_fn=create_model, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1, cv=3).fit(X_train, y_train)

print("The best score: %f using %s" % (grid.best_score_, grid.best_params_))
